### Titanic Dataset

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender = pd.read_csv('gender_submission.csv')

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
y = train[['Survived']]

# Let us just pick some features for X
X = train[['Sex','Age','Pclass']]
X = X.fillna(X.mean())
from sklearn.preprocessing import OneHotEncoder

ohc = OneHotEncoder()
ohc.fit(X[['Sex']])
data = pd.DataFrame(ohc.transform(X[['Sex']]).toarray())

X = pd.concat([X, data], axis=1)

X = X.drop(columns=['Sex'])
X = X.rename(columns={0:'Female',1: 'Male'})
X.head()

ohc.fit(y)
y = np.array(pd.DataFrame(ohc.transform(y).toarray()))
y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

Let us create a custom function to output our cleaned dataset in the format that we want

In [4]:
active = False

if active:
    index = 0
    training_data = []
    for i in np.array(X):
        training_data.append([i, y[index]])
        index+=1

    np.random.shuffle(training_data)
    np.save("titanic_train.npy", training_data)

In [5]:
training_data = np.load("titanic_train.npy", allow_pickle = True)
print(len(training_data))

891


### Making the Neural Network

In [17]:
training_data = np.load("titanic_train.npy", allow_pickle = True)

Let us create our Neural Network and call it `Titanic`. Being a beginner to Neural Networks myself, we will be using a simple linear fully connected layer with ReLu as our activation function. At the end, we will pass our results through a sigmoid function in order to get a range from 0 to 1.

In [7]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Titanic(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 2)
        self.fc4 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return torch.sigmoid(x)
    
net = Titanic()

So why did i choose a sigmoid function? Well, this is because the loss function we will be using is a Binary Cross Entropy Loss Function which takes in values ranging from 0 to 1

In [19]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.003)
loss_function = nn.BCELoss()
#loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data])

y = torch.Tensor([i[1] for i in training_data])

VALIDATION = 0.1 # Percent
validation_size = int(len(X) * VALIDATION)

train_X = X[:-validation_size]
train_y = y[:-validation_size]

test_X = X[-validation_size:]
test_y = y[-validation_size:]

print("Train:",len(train_X))
print("Test:",len(test_X))

Train: 802
Test: 89


`EPOCHS`: How many iterations

`BATCH_SIZE`: How much our step will be

In [22]:
BATCH_SIZE = 8
EPOCHS = 10

for epoch in range(EPOCHS):
    print("Epoch:", epoch)
    for i in range(0, len(train_X), BATCH_SIZE):
        batch_X = train_X[i:i+BATCH_SIZE]
        batch_y = train_y[i:i+BATCH_SIZE]
        
        net.zero_grad()
        output = net(batch_X)
        loss = loss_function(output, batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

Epoch: 0
tensor(0.2692, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 1
tensor(0.2586, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 2
tensor(0.2485, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 3
tensor(0.2383, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 4
tensor(0.2369, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 5
tensor(0.2326, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 6
tensor(0.2345, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 7
tensor(0.2239, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 8
tensor(0.2246, grad_fn=<BinaryCrossEntropyBackward>)
Epoch: 9
tensor(0.2232, grad_fn=<BinaryCrossEntropyBackward>)


In [23]:
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(test_X)):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,4))[0] # 0th element is net out
        
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total +=1
print("Accuracy:", round(correct/total, 3))

Accuracy: 0.831


### Validate our Model on the Test Set
With our Neural Network trained, let us validate this on the test set

In [24]:
# Processing the data
y = gender[['Survived']]

# Let us just pick some features for X
X = test[['Sex','Age','Pclass']]
X = X.fillna(X.mean())
from sklearn.preprocessing import OneHotEncoder

ohc = OneHotEncoder()
ohc.fit(X[['Sex']])
data = pd.DataFrame(ohc.transform(X[['Sex']]).toarray())

X = pd.concat([X, data], axis=1)

X = X.drop(columns=['Sex'])
X = X.rename(columns={0:'Female',1: 'Male'})
X.head()

ohc.fit(y)
y = np.array(pd.DataFrame(ohc.transform(y).toarray()))

In [12]:
active = False

if active:
    index = 0
    training_data = []
    for i in np.array(X):
        training_data.append([i, y[index]])
        index+=1

    np.random.shuffle(training_data)
    np.save("titanic_test.npy", training_data)

In [25]:
test_data = np.load('titanic_test.npy', allow_pickle = True)

test_X = torch.Tensor([i[0] for i in test_data])
test_y = torch.Tensor([i[1] for i in test_data])

In [26]:
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(test_X)):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,4))[0] # 0th element is net out
        
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total +=1
print("Accuracy:", round(correct/total, 3))

Accuracy: 0.959


In [27]:
test_data

tensor([[30.2726,  3.0000,  0.0000,  1.0000],
        [24.0000,  3.0000,  0.0000,  1.0000],
        [31.0000,  2.0000,  1.0000,  0.0000],
        ...,
        [26.0000,  3.0000,  1.0000,  0.0000],
        [30.2726,  3.0000,  0.0000,  1.0000],
        [18.5000,  3.0000,  1.0000,  0.0000]])